<a href="https://colab.research.google.com/github/urieliram/Pyomo_example/blob/main/UCP_Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 121874 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

\begin{align*}
\text{Min} \quad & \sum_{t\in T} \sum_{j\in J} ( c_{j} p_{jt} + c_{j}^\text{U} y_{jt}) \\
\text{Sujeto a:} \quad & \sum_{j\in J} p_{jt} = De_{t}, & t \in T \\
& \sum_{j\in J} \bar{p}_{jt} \geq De_{t} + R_{t}, &  t \in T\\
& c_{j}p_{jt} \geq \alpha_{js} p_{jt} + \beta_{js}, & s=1,…C_j, j \in J \\
& v_{j,t-1} - v_{jt} + y_{jt} - z_{jt} = 0 & j\in J, t \in T  \\
& p_{jt} - p_{j,t-1} \leq R_{j}^\text{U} v_{j,t-1} + S_{j}^\text{U} y_{jt} & j\in J, t \in T  \\
& p_{j,t-1} - p_{jt} \leq R_{j}^\text{D} v_{j,t} + S_{j}^\text{D} z_{jt} & j\in J, t \in T  \\
& \sum_{k=t-T_{j}^{U}+1,k \geq 1}^{t} y_{jk} \leq v_{jt} & j\in J, t\in [L_{j}+1,...,|T|] \\
& v_{jt}+\sum_{k=t-T_{j}^{D}+1,k \geq 1}^{t} z_{jk} \leq 1 & j\in J, t\in [F_{j}+1,...,|T|] \\
& P_{j}^{\text{min}} v_{jt} \leq p_{jt} \leq \bar{p}_{jt} \leq P_{j}^{\text{max}} v_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq p_{j,t-1} + R_{j}^{\text{U}} v_{j,t-1} + S_{j}^{\text{U}} Y_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq P_{j}^{\text{max}} (v_{jt}-z_{j,t+1})+ z_{j,t+1} S_{j}^{\text{D}} & j\in J, t \in T  \\
& P_{jt}, \bar{jt} \geq 0 \\
& v_{jt}, y_{jt}, z_{jt} = \{0,1\}
\end{align*}

Unit Commitment in Electric Energy Systems. M. F. Anjos and A. J. Conejo.  Foundations and Trends in Electric Energy Systems. Vol. 1, No. 4, 220--310 , 2017

In [120]:
T = {1,2,3,4,5,6}
G = {1,2,3}
Pmax = {1: 300, 2: 200, 3: 100}
Pmin = {1: 80, 2: 50, 3: 30}
cU = {1: 800, 2: 500, 3: 250}
c = {1: 5, 2: 15, 3: 30}
De = {1: 240, 2: 250, 3: 200, 4: 170, 5: 230, 6: 190}
R = {1: 0, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10}
RD = {1: 30, 2: 40, 3: 50}
RU = {1: 50, 2: 60, 3: 70}
SU = {1: 100, 2: 70, 3: 40}
SD = {1: 80, 2: 50, 3: 30}
TU = {1: 3, 2: 2, 3: 1}
TD = {1: 2, 2: 2, 3: 2}
U = {1: 2, 2: 0, 3: 0}
D = {1: 0, 2: 0, 3: 0}
v0 = {1: 1, 2: 0, 3: 0}
p0 = {1: 120, 2: 0, 3: 0}


In [136]:
from pyomo.environ import *
# Crear un modelo
model      = ConcreteModel()
# Conjuntos
model.G    = Set(initialize = G)
model.T    = Set(initialize = T)
# Parametros
model.Pmax  = Param(model.G    , initialize = Pmax  , within = Any)
model.Pmin  = Param(model.G    , initialize = Pmin  , within = Any)
model.c     = Param(model.G    , initialize = c     , within = Any)
model.cU    = Param(model.G    , initialize = cU    , within = Any)
model.De    = Param(model.T    , initialize = De    , within = Any)
model.R     = Param(model.T    , initialize = R     , within = Any)
model.RU    = Param(model.G    , initialize = RU    , within = Any)
model.RD    = Param(model.G    , initialize = RD    , within = Any)
model.SU    = Param(model.G    , initialize = SU    , within = Any)
model.SD    = Param(model.G    , initialize = SD    , within = Any)
model.U     = Param(model.G    , initialize = U     , within = Any)
model.D     = Param(model.G    , initialize = D     , within = Any)
model.TU    = Param(model.G    , initialize = TU    , within = Any)
model.TD    = Param(model.G    , initialize = TD    , within = Any)
model.v0    = Param(model.G    , initialize = v0   , within = Any)
model.p0    = Param(model.G    , initialize = p0   , within = Any)

# Variables
model.p     = Var( model.G , model.T , bounds = (0.0,99999.0))
model.pb    = Var( model.G , model.T , bounds = (0.0,99999.0))
model.v     = Var( model.G , model.T , within = Binary)
model.y     = Var( model.G , model.T , within = Binary)
model.z     = Var( model.G , model.T , within = Binary)



def obj_rule(m):
    return sum(( m.c[j] * m.p[j,t] + m.cU[j] * m.y[j,t] ) for t in m.T for j in m.G)
model.obj = Objective(rule = obj_rule,sense=minimize)

def demand_rule_1(m,t):                      ## demanda eq.(1)
    return sum( m.p[j,t] for j in m.G )   == m.De[t]
model.demand_rule_1 = Constraint(model.T, rule = demand_rule_1)

def demand_rule_2(m,t):                      ## demanda eq.(2)
    return sum( m.pb[j,t] for j in m.G )   >= m.De[t] + m.R[t]
model.demand_rule_2 = Constraint(model.T, rule = demand_rule_2)

## -----------------------------GARVER------------------------------------------
def logical_rule(m,j,t):     ## arranque-paro-operacion eq.(4)
    if t == 1:
        return  m.v0[j]   - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
    else:
        return m.v[j,t-1] - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
model.logical = Constraint(model.G,model.T,rule = logical_rule)

## Reservas
def ramping_up(m,j,t):
    if t == 1:
        return  m.p[j,t] - m.p0[j] <=  m.RU[j] * m.v0[j] + m.SU[j] * m.y[j,t]
    else:
        return   m.p[j,t] - m.p[j,t-1] <=  m.RU[j] * m.v[j,t-1] + m.SU[j] * m.y[j,t]
model.ramping_up = Constraint(model.G,model.T,rule = ramping_up)
def ramping_down(m,j,t):
    if t == 1:
        return m.p0[j] - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
    else:
        return m.p[j,t-1] - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
model.ramping_down = Constraint(model.G,model.T,rule = ramping_down)

## Limites de los generadores
def limite_uno(m,j,t):
    return m.Pmin[ j ] * m.v[ j , t ] <= m.p[ j , t ]
model.limite_uno = Constraint(model.G,model.T,rule = Limite_uno)

def limite_dos(m,j,t):
    return m.p[ j , t ] <= m.pb[ j , t ]
model.limite_dos = Constraint(model.G,model.T,rule = Limite_dos)

def limite_tres(m,j,t):
    return m.pb[ j , t ] <= m.Pmax[ j ] * m.v[ j , t ]
model.limite_tres = Constraint(model.G,model.T,rule = Limite_tres)

def limite_arranque(m,j,t):
    return m.pb[ j , t ] <= m.p0[ j ] + m.RU[ j ] * m.v0[ j ] + m.SU[ j ] * m.y[ j , t]
model.limite_arranque = Constraint(model.G,model.T,rule = limite_arranque)


# Uptime    * * * * * * * * * * * *
  subject to Const_Uptime{ j in J , t in (min(card(T),U[j])+1)..(card(T)) } :
  sum{ k in ( t-TU[j]+1)..t:k>=1 } y[ j , k ] <=  v[ j , t ];

# Downtime  * * * * * * * * * * * *
  subject to Const_Downtime{j in J , t in (min(card(T),D[j])+1)..(card(T)) } :
  v[ j , t ] + sum{ k in ( t-TD[j]+1)..t:k>=1 } z[ j , k ] <= 1 ;


# Generation Limits 4
  subject to Const_GenerationLimits4{ j in J , t in T:t>1} :
  pb[ j , t ] <= p[ j , t - 1 ] + RU[ j ] * v[ j , t - 1 ] + SU[ j ] * y[ j , t ];

# Generation Limits 5
  subject to Const_GenerationLimits5{ j in J , t in T:(t<(card(T)-1))} :
  pb[ j , t ] <= Pmax[ j ] * ( v[j,t] - z[j,t+1] ) + z[j,t+1] * SD[ j ];

# model.pprint() ## Se imprime el modelo

In [134]:
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

print('\n z = ', model.obj())

print('\nDecision Variables')
print('p = ', model.p[1,1].value)
print('pb = ', model.pb)
print('v = ', model.v)
print('y = ', model.y)
print('z = ', model.z)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8700.0
  Upper bound: 8700.0
  Number of objectives: 1
  Number of constraints: 70
  Number of variables: 64
  Number of binary variables: 54
  Number of integer variables: 54
  Number of nonzeros: 30
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.49
  Wallclock time: 0.5
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and boun